# IMPORTS

In [ ]:
import torch
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch.nn as nn
from torch.nn.modules.module import Module
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR,StepLR

import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import sys

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import  argparse

args = argparse.ArgumentParser()
args.add_argument('--dataset', default='cora')
args.add_argument('--model', default='gcn')
args.add_argument('--learning_rate', type=float, default=0.01)
args.add_argument('--epochs', type=int, default=400)
args.add_argument('--hidden', type=int, default=16)
args.add_argument('--dropout', type=float, default=0.5)
args.add_argument('--weight_decay', type=float, default=5e-4)
args.add_argument('--early_stopping', type=int, default=10)
args.add_argument('--max_degree', type=int, default=3)
args.add_argument('-f')

args = args.parse_args()

# DATASET

In [ ]:
def sparse_to_tuple(sparse_mx):
  """
  Convert sparse matrix to tuple representation.
  """
  def to_tuple(mx):
      if not sp.isspmatrix_coo(mx):
          mx = mx.tocoo()
      coords = np.vstack((mx.row, mx.col)).transpose()
      values = mx.data
      shape = mx.shape
      return coords, values, shape

  if isinstance(sparse_mx, list):
      for i in range(len(sparse_mx)):
          sparse_mx[i] = to_tuple(sparse_mx[i])
  else:
      sparse_mx = to_tuple(sparse_mx)

  return sparse_mx


def preprocess_features(features):
  """
  Row-normalize feature matrix and convert to tuple representation
  """
  rowsum = np.array(features.sum(1)) # get sum of each row, [2708, 1]
  r_inv = np.power(rowsum, -1).flatten() # 1/rowsum, [2708]
  r_inv[np.isinf(r_inv)] = 0. # zero inf data
  r_mat_inv = sp.diags(r_inv) # sparse diagonal matrix, [2708, 2708]
  features = r_mat_inv.dot(features) # D^-1:[2708, 2708]@X:[2708, 2708]
  return sparse_to_tuple(features) # [coordinates, data, shape], []


def normalize_adj(adj):
  """Symmetrically normalize adjacency matrix."""
  adj = sp.coo_matrix(adj)
  rowsum = np.array(adj.sum(1)) # D
  d_inv_sqrt = np.power(rowsum, -0.5).flatten() # D^-0.5
  d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
  d_mat_inv_sqrt = sp.diags(d_inv_sqrt) # D^-0.5
  return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # D^-0.5AD^0.5


def preprocess_adj(adj):
  """Preprocessing of adjacency matrix for simple GCN model and conversion to tuple representation."""
  adj_normalized = normalize_adj(adj + sp.eye(adj.shape[0]))
  return sparse_to_tuple(adj_normalized)

def sparse_dropout(x, rate, noise_shape):
  """
  :param x:
  :param rate:
  :param noise_shape: int scalar
  :return:
  """
  random_tensor = 1 - rate
  random_tensor += torch.rand(noise_shape).to(x.device)
  dropout_mask = torch.floor(random_tensor).byte()
  i = x._indices() # [2, 49216]
  v = x._values() # [49216]

  # [2, 4926] => [49216, 2] => [remained node, 2] => [2, remained node]
  i = i[:, dropout_mask]
  v = v[dropout_mask]

  out = torch.sparse.FloatTensor(i, v, x.shape).to(x.device)

  out = out * (1./ (1-rate))

  return out

def masked_loss(out, label, mask):

  loss = F.cross_entropy(out, label, reduction='none')
  mask = mask.float()
  mask = mask / mask.mean()
  loss *= mask
  loss = loss.mean()
  return loss


def masked_acc(out, label, mask):
  # [node, f]
  pred = out.argmax(dim=1)
  correct = torch.eq(pred, label).float()
  mask = mask.float()
  mask = mask / mask.mean()
  correct *= mask
  acc = correct.mean()
  return acc

In [ ]:
def parse_index_file(filename):
  """
  Parse index file.
  """
  index = []
  for line in open(filename):
      index.append(int(line.strip()))
  return index

def sample_mask(idx, l):
  """
  Create mask.
  """
  mask = np.zeros(l)
  mask[idx] = 1
  return np.array(mask, dtype=np.bool)

def load_data(dataset_str):
  """
  Loads input data from gcn/data directory
  ind.dataset_str.x => the feature vectors of the training instances as scipy.sparse.csr.csr_matrix object;
  ind.dataset_str.tx => the feature vectors of the test instances as scipy.sparse.csr.csr_matrix object;
  ind.dataset_str.allx => the feature vectors of both labeled and unlabeled training instances
      (a superset of ind.dataset_str.x) as scipy.sparse.csr.csr_matrix object;
  ind.dataset_str.y => the one-hot labels of the labeled training instances as numpy.ndarray object;
  ind.dataset_str.ty => the one-hot labels of the test instances as numpy.ndarray object;
  ind.dataset_str.ally => the labels for instances in ind.dataset_str.allx as numpy.ndarray object;
  ind.dataset_str.graph => a dict in the format {index: [index_of_neighbor_nodes]} as collections.defaultdict
      object;
  ind.dataset_str.test.index => the indices of test instances in graph, for the inductive setting as list object.
  All objects above must be saved using python pickle module.
  :param dataset_str: Dataset name
  :return: All data input files loaded (as well the training/test data).
  """
  names = ['x', 'y', 'tx', 'ty', 'allx', 'ally', 'graph']
  objects = []
  for i in range(len(names)):
    with open("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.{}".format(dataset_str, names[i]), 'rb') as f:
      if sys.version_info > (3, 0):
        objects.append(pkl.load(f, encoding='latin1'))
      else:
        objects.append(pkl.load(f))

  x, y, tx, ty, allx, ally, graph = tuple(objects)
  test_idx_reorder = parse_index_file("/content/drive/MyDrive/THESIS/Databases/data/ind.{}.test.index".format(dataset_str))
  test_idx_range = np.sort(test_idx_reorder)

  if dataset_str == 'citeseer':
    # Fix citeseer dataset (there are some isolated nodes in the graph)
    # Find isolated nodes, add them as zero-vecs into the right position
    test_idx_range_full = range(min(test_idx_reorder), max(test_idx_reorder)+1)
    tx_extended = sp.lil_matrix((len(test_idx_range_full), x.shape[1]))
    tx_extended[test_idx_range-min(test_idx_range), :] = tx
    tx = tx_extended
    ty_extended = np.zeros((len(test_idx_range_full), y.shape[1]))
    ty_extended[test_idx_range-min(test_idx_range), :] = ty
    ty = ty_extended

  features = sp.vstack((allx, tx)).tolil()
  features[test_idx_reorder, :] = features[test_idx_range, :]
  adj = nx.adjacency_matrix(nx.from_dict_of_lists(graph))

  labels = np.vstack((ally, ty))
  labels[test_idx_reorder, :] = labels[test_idx_range, :]

  idx_test = test_idx_range.tolist()
  idx_train = range(len(y))
  idx_val = range(len(y), len(y)+500)

  train_mask = sample_mask(idx_train, labels.shape[0])
  val_mask = sample_mask(idx_val, labels.shape[0])
  test_mask = sample_mask(idx_test, labels.shape[0])

  y_train = np.zeros(labels.shape)
  y_val = np.zeros(labels.shape)
  y_test = np.zeros(labels.shape)
  y_train[train_mask, :] = labels[train_mask, :]
  y_val[val_mask, :] = labels[val_mask, :]
  y_test[test_mask, :] = labels[test_mask, :]

  return adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask

In [ ]:
adj, features, y_train, y_val, y_test, train_mask, val_mask, test_mask = load_data(args.dataset)

In [ ]:
features = preprocess_features(features) # [coordinates, data, shape]
A_hats = preprocess_adj(adj)

In [ ]:
# should send all the info to cuda
train_label = torch.from_numpy(y_train).long()
num_classes = train_label.shape[1]
train_label = train_label.argmax(dim=1)
train_mask = torch.from_numpy(train_mask.astype(np.int))
val_label = torch.from_numpy(y_val).long()
val_label = val_label.argmax(dim=1)
val_mask = torch.from_numpy(val_mask.astype(np.int))
test_label = torch.from_numpy(y_test).long()
test_label = test_label.argmax(dim=1)
test_mask = torch.from_numpy(test_mask.astype(np.int))

In [ ]:
i = torch.from_numpy(features[0]).long()
v = torch.from_numpy(features[1])
feature = torch.sparse.FloatTensor(i.t(), v, features[2])

i = torch.from_numpy(A_hats[0]).long()
v = torch.from_numpy(A_hats[1])
A_hat = torch.sparse.FloatTensor(i.t(), v, A_hats[2]).float()

In [ ]:
num_features_nonzero = feature._nnz()
feat_dim = feature.shape[1]

# MODELS

In [ ]:
class GraphConvolution(nn.Module):

  def __init__(self, input_dim, output_dim, num_features_nonzero, dropout=0., is_sparse_inputs=False, bias=False, activation = F.relu):
    super(GraphConvolution, self).__init__()


    self.dropout = dropout
    self.bias = bias
    self.activation = activation
    self.is_sparse_inputs = is_sparse_inputs
    self.num_features_nonzero = num_features_nonzero

    self.weight = nn.Parameter(torch.randn(input_dim, output_dim))
    self.bias = None
    if bias:
      self.bias = nn.Parameter(torch.zeros(output_dim))
    
  def forward(self, inputs):
    x, adj = inputs
    if self.training and self.is_sparse_inputs:
      x = sparse_dropout(x, self.dropout, self.num_features_nonzero)

    if self.is_sparse_inputs:
      xw = torch.sparse.mm(x, self.weight)
    else:
      xw = torch.mm(x, self.weight)
    out = torch.sparse.mm(adj, xw)

    if self.bias is not None:
      out += self.bias

    return self.activation(out), adj


In [ ]:
class GCN(nn.Module):

  def __init__(self, input_dim, output_dim, num_features_nonzero):
    super(GCN, self).__init__()

    self.input_dim = input_dim # 1433
    self.output_dim = output_dim

    print('input dim:', input_dim)
    print('output dim:', output_dim)
    print('num_features_nonzero:', num_features_nonzero)

    self.layers = nn.Sequential(GraphConvolution(input_dim, args.hidden, num_features_nonzero,
                                                     activation=F.relu,
                                                     dropout=args.dropout,
                                                     is_sparse_inputs=True),

                                GraphConvolution(args.hidden, output_dim, num_features_nonzero,
                                                     activation=F.relu,
                                                     dropout=args.dropout,
                                                     is_sparse_inputs=False),

                             )
  def forward(self, inputs):

    x, adj = inputs

    x = self.layers((x, adj))
    return x
  
  def l2_loss(self): 
    layer = self.layers.children() 
    layer = next(iter(layer)) 
    loss = None 
    for layer in self.layers: 
      for p in layer.parameters(): 
        if loss is None: 
          loss = p.pow(2).sum() 
        else: 
          loss += p.pow(2).sum() 
    return loss
  

In [ ]:
net = GCN(feat_dim, num_classes, num_features_nonzero)
optimizer = optim.Adam(net.parameters(), lr=args.learning_rate)

input dim: 1433
output dim: 7
num_features_nonzero: 49216


In [ ]:
net.train()
for epoch in range(args.epochs):
  out = net((feature, A_hat))
  out = out[0]

  loss = masked_loss(out, train_label, train_mask)
  loss += args.weight_decay * net.l2_loss()

  acc = masked_acc(out, train_label, train_mask)
  acc_val = masked_acc(out, val_label, val_mask)
  acc_test = masked_acc(out, test_label, test_mask)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if epoch % 10 == 0:

    print(epoch, loss.item(), acc.item(), acc_val.item(), acc_test.item())

net.eval()

out = net((feature, A_hat))
out = out[0]
acc = masked_acc(out, test_label, test_mask)
print('test:', acc.item())


0 13.632501602172852 0.12142856419086456 0.12600000202655792 0.13199999928474426


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:64: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  ../aten/src/ATen/native/IndexingUtils.h:30.)


10 11.910115242004395 0.12857143580913544 0.14000000059604645 0.1459999829530716
20 10.37842082977295 0.19999998807907104 0.18999998271465302 0.1810000091791153
30 9.081487655639648 0.25 0.19999998807907104 0.21799999475479126
40 7.981378555297852 0.30714282393455505 0.21199999749660492 0.2369999885559082
50 7.042667865753174 0.3999999761581421 0.2459999918937683 0.2619999945163727
60 6.218443870544434 0.4642857015132904 0.27399998903274536 0.2959999740123749
70 5.512348175048828 0.5071429014205933 0.2879999876022339 0.3190000057220459
80 4.915709495544434 0.550000011920929 0.37999996542930603 0.38599997758865356
90 4.3901262283325195 0.7000000476837158 0.42799997329711914 0.44699999690055847
100 3.8890771865844727 0.7500000596046448 0.4819999933242798 0.4899999797344208
110 3.469174861907959 0.7571429014205933 0.5080000162124634 0.5210000872612
120 3.0877814292907715 0.8071427941322327 0.5579999685287476 0.5679999589920044
130 2.7454276084899902 0.8571428656578064 0.5759999752044678 0